In [168]:
#Import necessary dependencies. The last dependency is a style format for matplot lib graphs that will done towards the end of this script. 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests 
import csv 
import seaborn as sb

In [169]:
# Per instructions we needed to create a file path for our output data to be stored. What should appear in this file by the end of this code will be a data frame of the information collected
City_data_file = "Resources/City_Data.csv"

In [170]:
# need to import citipy to help tag geo coordinates to cities selected
from citipy import citipy 

In [171]:
# import my individualized API key and define desired units for temperature because I will need to successfuly call url 
from BK_api_key import BK_api_key

temperature_units = "imperial"



In [172]:
# Prior to any list creation or defining range for specific values we will look for, I need make sure my query_url works. Base url was provided from OpenWeatherMap recommendations on how to perform successful API call 
url_query = f"http://api.openweathermap.org/data/2.5/weather?appid={BK_api_key}&units={temperature_units}&q="

In [173]:
# Since we will be capturing the lat and long of the random cities I will need define ranges for those values and create an open list to add those values later on. Range values are based on researched max and min values for lat and long  

latitude_range = (-90, 90)
longitude_range = (-180, 180)


latit_and_longtit_list = []
cities = []


In [174]:
#From starter code
# Need to use np module to generate selection of the random cities and for those to be stored in the open lists I created above. Obviously what's different is how I named my variables.
latitude_values = np.random.uniform(low=-90.000, high=90.000, size=1500)
longitude_values = np.random.uniform(low=-180.000, high=180.000, size=1500)

latit_and_longtit_list = zip(latitude_values, longitude_values)

# For loop that will help identify nearest city (using citipy) for each latitude and longitude combination found
for latit_and_longtit_list in latit_and_longtit_list:
    city = citipy.nearest_city(latit_and_longtit_list[0], latit_and_longtit_list[1]).city_name
    
    # Conditional statement to ensure that only unquie city values are added to our open cities list
    if city not in cities:
        cities.append(city)

    # Print the city count to confirm sufficient count
len(cities)

604

In [175]:
#Double check to ensure actual city values were gathered from citipy module.
print(cities)

['hilo', 'svetlogorsk', 'san jose', 'codrington', 'shkotovo-26', 'rehoboth', 'busselton', 'burica', 'bluff', 'mys shmidta', 'ushuaia', 'puerto ayora', 'sheridan', 'datong', 'palana', 'khatanga', 'tasiilaq', 'qaanaaq', 'joshimath', 'bethel', 'cabo san lucas', 'klaksvik', 'albany', 'zlobin', 'castlegar', 'saint-georges', 'cherskiy', 'vaini', 'hermanus', 'rikitea', 'ancud', 'termiz', 'atuona', 'tuktoyaktuk', 'sol-iletsk', 'bredasdorp', 'tarana', 'ilulissat', 'rio gallegos', 'bedele', 'cape town', 'kodiak', 'barrow', 'esso', 'javanrud', 'mataura', 'afmadu', 'illoqqortoormiut', 'fort-shevchenko', 'talnakh', 'east london', 'khonsa', 'saint george', 'kapaa', 'warmbad', 'rawah', 'ialibu', 'bracebridge', 'ninohe', 'poya', 'marcona', 'saint-philippe', 'stromness', 'taolanaro', 'zhuhai', 'padang', 'saskylakh', 'katete', 'pisco', 'katsuura', 'attawapiskat', 'bulgan', 'high level', 'new norfolk', 'teya', 'mandalgovi', 'muisne', 'doctor pedro p. pena', 'ozernovskiy', 'altay', 'tabiauea', 'taoudenni'

In [176]:
# I then created an initial data frame for these cities to be stored in. 
Project_Primary_df = pd.DataFrame(cities)

In [177]:
# Further breakdown of the data frame I needed to create to store the list of cities gathered from the citipy command. I first decided to choose a sample of 600 cities because the minimum requirement was 500. I think reset the index for the data frame to the different cities that were collected in the list. From there I just renamed each of the column values to required fileds of information that we would be calling from the API. Performed a .head() to ensure that my data frame was set up correctly. 
Project_Secondary_df = Project_Primary_df.sample(600)
Project_Secondary_df = Project_Secondary_df.reset_index(drop = True)
Project_Secondary_df.columns = ["City"]
Project_Secondary_df.index += 1 
Project_Secondary_df["Latitudes"] = ""
Project_Secondary_df["Longitudes"] = ""
Project_Secondary_df["Country"] = ""
Project_Secondary_df["Date"] = ""
Project_Secondary_df["Temp Max"] = ""
Project_Secondary_df["Humidity"] = ""
Project_Secondary_df["Cloudiness"] = ""
Project_Secondary_df["Wind Speed"] = ""
Project_Secondary_df.head()

,City,Latitudes,Longitudes,Country,Date,Temp Max,Humidity,Cloudiness,Wind Speed
1,balimo,,,,,,,,
2,olafsvik,,,,,,,,
3,alta floresta,,,,,,,,
4,riachao das neves,,,,,,,,
5,east london,,,,,,,,


In [178]:
# After multiple failed attempts in trying to run a for loop for 'city in cities', I was able to figure out that I could run a for loop that was predicated on the index set in the data frame above. From there the loop would run the iterative commands to collect each of the specific data points I needed from the API call. I was able to figure out what tags from the API call to put in my code by first running the call in json on the first couple of cities. One thing I did have to research was how to remove the spaces ("") from my json script. 
#The only thing I was unable to do was to group my processing records in differents sets of 50 like the exemplar. Instead I printed off my processing records based off my index values which I preset in the data frame above. 
for index,row in Project_Secondary_df.iterrows():
    try: 
        city_names = row["City"]
        response = requests.get(url_query + city_names.replace(" ","%20")).json()
        Project_Secondary_df.set_value(index,"Latitudes",response.get("coord",{}).get("lat"))
        Project_Secondary_df.set_value(index,"Longitudes",response.get("coord",{}).get("lon"))
        Project_Secondary_df.set_value(index,"Country",response.get("sys",{}).get("country"))
        Project_Secondary_df.set_value(index,"Date",response.get("dt",{}))
        Project_Secondary_df.set_value(index,"Temp Max",response.get("main",{}).get("temp_max"))
        Project_Secondary_df.set_value(index,"Humidity",response.get("main",{}).get("humidity"))
        Project_Secondary_df.set_value(index,"Cloudiness",response.get("clouds",{}).get("all"))
        Project_Secondary_df.set_value(index,"Wind Speed",response.get("wind",{}).get("speed"))
        print("Processing Record " + str(index) + " of 600 | " + str(city_names))
    except Exception:
        print("City not found. Skipping...")
print("-------------------------\nData Retrieval Complete\n-------------------------")


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __nam

Processing Record 1 of 600 | balimo
Processing Record 2 of 600 | olafsvik
Processing Record 3 of 600 | alta floresta
Processing Record 4 of 600 | riachao das neves
Processing Record 5 of 600 | east london
Processing Record 6 of 600 | mocambique
Processing Record 7 of 600 | thalassery
Processing Record 8 of 600 | hohhot
Processing Record 9 of 600 | kavieng
Processing Record 10 of 600 | guadalajara
Processing Record 11 of 600 | lanigan
Processing Record 12 of 600 | saleaula
Processing Record 13 of 600 | zlobin
Processing Record 14 of 600 | lagoa
Processing Record 15 of 600 | pevek
Processing Record 16 of 600 | ushuaia
Processing Record 17 of 600 | coihaique
Processing Record 18 of 600 | te anau
Processing Record 19 of 600 | fort nelson
Processing Record 20 of 600 | ironton
Processing Record 21 of 600 | ternate
Processing Record 22 of 600 | fort madison
Processing Record 23 of 600 | san carlos de bariloche
Processing Record 24 of 600 | henties bay
Processing Record 25 of 600 | louisbourg


Processing Record 208 of 600 | warmbad
Processing Record 209 of 600 | lorengau
Processing Record 210 of 600 | los llanos de aridane
Processing Record 211 of 600 | manaure
Processing Record 212 of 600 | yuksekova
Processing Record 213 of 600 | hofn
Processing Record 214 of 600 | buqayq
Processing Record 215 of 600 | bundaberg
Processing Record 216 of 600 | khatanga
Processing Record 217 of 600 | atuona
Processing Record 218 of 600 | egvekinot
Processing Record 219 of 600 | iqaluit
Processing Record 220 of 600 | amudalavalasa
Processing Record 221 of 600 | jurado
Processing Record 222 of 600 | trinidad
Processing Record 223 of 600 | bengkulu
Processing Record 224 of 600 | hagenow
Processing Record 225 of 600 | tumannyy
Processing Record 226 of 600 | batesville
Processing Record 227 of 600 | jaisalmer
Processing Record 228 of 600 | nome
Processing Record 229 of 600 | kahului
Processing Record 230 of 600 | turukhansk
Processing Record 231 of 600 | poya
Processing Record 232 of 600 | tungor

Processing Record 411 of 600 | tura
Processing Record 412 of 600 | walvis bay
Processing Record 413 of 600 | kaputa
Processing Record 414 of 600 | madison
Processing Record 415 of 600 | hilo
Processing Record 416 of 600 | new norfolk
Processing Record 417 of 600 | saldanha
Processing Record 418 of 600 | marsh harbour
Processing Record 419 of 600 | santa cruz
Processing Record 420 of 600 | honiara
Processing Record 421 of 600 | iralaya
Processing Record 422 of 600 | hualmay
Processing Record 423 of 600 | richards bay
Processing Record 424 of 600 | opuwo
Processing Record 425 of 600 | kasempa
Processing Record 426 of 600 | ialibu
Processing Record 427 of 600 | skibbereen
Processing Record 428 of 600 | manzhouli
Processing Record 429 of 600 | hasaki
Processing Record 430 of 600 | sol-iletsk
Processing Record 431 of 600 | prieska
Processing Record 432 of 600 | zhetybay
Processing Record 433 of 600 | princeton
Processing Record 434 of 600 | bathsheba
Processing Record 435 of 600 | raga
Proc

In [184]:
#Now that my data was able to successfully be pulled from the API for the specific cities I pulled from the citipy module, I needed to transfer this data to the output file saved towards the beginning of my cound. 
#Performed a .count to ensure that same number of values were present for each column. The axis = 0 allows for this information to presented in a stacked layout that is better visually than the unstacked version. 
Project_Secondary_df.to_csv("City_Data.csv")
Project_Secondary_df.count(axis=0)


City          550
Latitudes     550
Longitudes    550
Country       550
Date          550
Temp Max      550
Humidity      550
Cloudiness    550
Wind Speed    550
dtype: int64

In [182]:
#Performed .head() to capture preview of completed data frame, using the data pulled from the successful API call done in my code. 
Project_Secondary_df.head()

,City,Latitudes,Longitudes,Country,Date,Temp Max,Humidity,Cloudiness,Wind Speed
2,olafsvik,64.89,-23.71,IS,1581367930,33.48,70,76,23.33
3,alta floresta,-9.88,-56.09,BR,1581367865,89.6,55,20,3.36
4,riachao das neves,-11.75,-44.91,BR,1581367930,76.5,91,97,2.95
5,east london,-33.02,27.91,ZA,1581367930,71.31,68,35,9.64
7,thalassery,11.75,75.53,IN,1581367930,73.4,94,20,3.36
